In [2]:
import os
import requests
import json
import tensorflow as tf

from modules.utils import (
    make_serialize_example, 
    get_example_from_uri, 
    decode_example, 
    make_base64_example
)
from pprint import PrettyPrinter
pp = PrettyPrinter()

MODEL_NAME='review-sentiment-model'
IP_ADDRESS='103.190.215.213:8501'

base_url = 'http://' + IP_ADDRESS + '/v1/models/' + MODEL_NAME

## Test docker and request metadata

In [6]:
pp.pprint(requests.get(base_url + '/metadata').json())

{'metadata': {'signature_def': {'signature_def': {'__saved_model_init_op': {'defaults': {},
                                                                            'inputs': {},
                                                                            'method_name': '',
                                                                            'outputs': {'__saved_model_init_op': {'dtype': 'DT_INVALID',
                                                                                                                  'name': 'NoOp',
                                                                                                                  'tensor_shape': {'dim': [],
                                                                                                                                   'unknown_rank': True}}}},
                                                  'serving_default': {'defaults': {},
                                                                      '

## Make TF Serving prediction

In [7]:
# Load test data from exampleGen split eval
test_uri = os.path.join('output/chahya_santoso-pipeline/CsvExampleGen/examples/1', 'Split-eval')
serialized_examples = get_example_from_uri(test_uri,1000)
#print(serialized_examples[:5])
features = {
    'review': tf.io.VarLenFeature(tf.string),
    'sentiment': tf.io.VarLenFeature(tf.string),
}

reviews, sentiments = decode_example(serialized_examples, features)
reviews = [review.decode() for review in reviews]
sentiments = [sentiment.decode() for sentiment in sentiments]
print(reviews[:5])
print(sentiments[:5])

["This show was an amazing, fresh & innovative idea in the 70's when it first aired. The first 7 or 8 years were brilliant, but things dropped off after that. By 1990, the show was not really funny anymore, and it's continued its decline further to the complete waste of time it is today.<br /><br />It's truly disgraceful how far this show has fallen. The writing is painfully bad, the performances are almost as bad - if not for the mildly entertaining respite of the guest-hosts, this show probably wouldn't still be on the air. I find it so hard to believe that the same creator that hand-selected the original cast also chose the band of hacks that followed. How can one recognize such brilliance and then see fit to replace it with such mediocrity? I felt I must give 2 stars out of respect for the original cast that made this show such a huge success. As it is now, the show is just awful. I can't believe it's still on the air.", 'If you like original gut wrenching laughter you will like th

In [8]:
base64_examples = [make_base64_example(example) for example in serialized_examples]
json_data = json.dumps(
    {
        'instances': base64_examples
    }
)

response = requests.post(base_url + ':predict', data=json_data)

#pp.pprint(response.json())
if 'predictions' in response.json():
    output = response.json()['predictions']
    y_true = sentiments

    y_pred = [row['output'] for row in output]
    y_pred_sentiment = [row['label'] for row in output]

    for i, review in enumerate(reviews):
        print('Review:', review)
        print('Real sentiment:', y_true[i])
        print('Prediction sentiment:', y_pred_sentiment[i], ', Score:', y_pred[i], '\n')
else:
    pp.pprint(response.json())



Review: This show was an amazing, fresh & innovative idea in the 70's when it first aired. The first 7 or 8 years were brilliant, but things dropped off after that. By 1990, the show was not really funny anymore, and it's continued its decline further to the complete waste of time it is today.<br /><br />It's truly disgraceful how far this show has fallen. The writing is painfully bad, the performances are almost as bad - if not for the mildly entertaining respite of the guest-hosts, this show probably wouldn't still be on the air. I find it so hard to believe that the same creator that hand-selected the original cast also chose the band of hacks that followed. How can one recognize such brilliance and then see fit to replace it with such mediocrity? I felt I must give 2 stars out of respect for the original cast that made this show such a huge success. As it is now, the show is just awful. I can't believe it's still on the air.
Real sentiment: negative
Prediction sentiment: negative ,

In [9]:
from sklearn.metrics import confusion_matrix, classification_report
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred_sentiment))
print('\nClassification Report')
print(classification_report(y_true, y_pred_sentiment))

Confusion Matrix
[[456  44]
 [ 61 439]]

Classification Report
              precision    recall  f1-score   support

    negative       0.88      0.91      0.90       500
    positive       0.91      0.88      0.89       500

    accuracy                           0.90      1000
   macro avg       0.90      0.90      0.89      1000
weighted avg       0.90      0.90      0.89      1000



In [24]:
# Test 2 - use my own data
reviews = [
    "I love this show so far and the acting is cool",
    "the movie is good",
    "the movie is not good, it's trash",
    "movie is bad",
    "The movie is terrible"
    ]
sentiments = [
    'positive',
    'positive',
    'negative',
    'negative',
    'negative'
    ]

serialized_examples = [
    make_serialize_example({
        'review':review, 
        'sentiment':sentiments[index]
    }) for index, review in enumerate(reviews)
]

In [25]:
base64_examples = [make_base64_example(example) for example in serialized_examples]
json_data = json.dumps(
    {
        'instances': base64_examples
    }
)

response = requests.post(base_url + ':predict', data=json_data)

#pp.pprint(response.json())
if 'predictions' in response.json():
    output = response.json()['predictions']
    y_true = sentiments

    y_pred = [row['output'] for row in output]
    y_pred_sentiment = [row['label'] for row in output]

    for i, review in enumerate(reviews):
        print('Review:', review)
        print('Real sentiment:', y_true[i])
        print('Prediction sentiment:', y_pred_sentiment[i], ', Score:', y_pred[i], '\n')
else:
    pp.pprint(response.json())

Review: I love this show so far and the acting is cool
Real sentiment: positive
Prediction sentiment: positive , Score: [0.60069418] 

Review: the movie is good
Real sentiment: positive
Prediction sentiment: positive , Score: [0.589530706] 

Review: the movie is not good, it's trash
Real sentiment: negative
Prediction sentiment: negative , Score: [0.435306609] 

Review: movie is bad
Real sentiment: negative
Prediction sentiment: negative , Score: [0.23010993] 

Review: The movie is terrible
Real sentiment: negative
Prediction sentiment: negative , Score: [0.110575169] 



In [26]:
from sklearn.metrics import confusion_matrix, classification_report
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred_sentiment))
print('\nClassification Report')
print(classification_report(y_true, y_pred_sentiment))

Confusion Matrix
[[3 0]
 [0 2]]

Classification Report
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         3
    positive       1.00      1.00      1.00         2

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5

